In [53]:
%%HTML


<style type="text/css">
<link href="https://fonts.googleapis.com/css2?family=Poppins&display=swap" rel="stylesheet">

* {
    font-family: 'Poppins', sans-serif;
}
div.summary {
    background-color: darkorange; 
    color: white; 
    padding: 8px; 
    padding-right: 300px; 
    font-size: 24px; 
    max-width: 1500px; 
    margin-top: 50px;
    margin-bottom:4px;
    border-radius: 5px;
 }

 div.chapter { 
    color: darkorange; 
    padding: 8px; 
    padding-right: 300px; 
    font-size: 24px; 
    max-width: 1500px; 
    margin-top: 50px;
    margin-bottom:4px;
 }

  div.sub-chapter { 
    color: darkorange; 
    padding: 8px; 
    padding-right: 300px; 
    font-size: 20px; 
    max-width: 1500px; 
    margin-top: 30px;
    margin-bottom:4px;
 }

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from IPython.core.display import HTML

%matplotlib inline

In [47]:
def get_distinct_vals(df):
    return df.agg(*(countDistinct(col(c)).alias(c) for c in df.columns)).toPandas().T.reset_index().rename(columns={'index':'column', 0:'n_distinct'})

In [50]:
def get_missing_vals_cnt(df):
    return df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas().T.reset_index().rename(columns={'index':'column', 0:'n_missing'})

In [61]:
def get_duplicate_rows(df):
    return df.groupBy(df.columns).count().filter(col("count") > 1).collect()

# <center>An Exploratory Analysis of The NFL Big Data Bowl</center>
## <center>Using Python & Pyspark</center></div>

In [4]:
%%HTML
<center><img src='../static/nfl-data-bowl.png', align='center'></center>

<div class=summary>1. Summary</div>

* Is there any way to use player tracking data to predict whether or not certain penalties – for example, defensive pass interference – will be called?

In [6]:
# initialise spark session
sc = SparkSession.builder.appName('nfl-eda')\
    .master("local[15]").getOrCreate()

<div class=chapter>2. Exploratory Analysis</div>
<p>The raw data consisted of multiple different datasets. The descriptions (as listed on Kaggle) of these datasets can be found below:</p>
<ul>
    <li>Game data: The games.csv contains the teams playing in each game. The key variable is gameId.</li>
    <li>Player data: The players.csv file contains player-level information from players that participated in any of the tracking data files. The key variable is nflId.</li>
    <li>Play data: The plays.csv file contains play-level information from each game. The key variables are gameId and playId.</li>
    <li>Tracking data: Files week[week].csv contain player tracking data from all games in week [week]. The key variables are gameId, playId, and nflId. There are 17 weeks to a typical NFL Regular Season, and thus 17 data frames with player tracking data are provided.</li>
</ul>
<p>The following sub-sections describes the exploratory analysis that was applied to these datasets.</p>
<div class=sub-chapter>2.1 Game Data</div>

In [55]:
# read in games
games_df = sc.read.csv("../data/games.csv", header=True, inferSchema=True)
games_df.printSchema()

root
 |-- gameId: integer (nullable = true)
 |-- gameDate: string (nullable = true)
 |-- gameTimeEastern: string (nullable = true)
 |-- homeTeamAbbr: string (nullable = true)
 |-- visitorTeamAbbr: string (nullable = true)
 |-- week: integer (nullable = true)



In [58]:
missing_vals = get_missing_vals_cnt(games_df)
distinct_vals = get_distinct_vals(games_df)
missing_vals.merge(distinct_vals, on="column", how="inner")

,column,n_missing,n_distinct
0,gameId,0,253
1,gameDate,0,50
2,gameTimeEastern,0,10
3,homeTeamAbbr,0,32
4,visitorTeamAbbr,0,32
5,week,0,17


In [69]:
games_df.summary().toPandas()

,summary,gameId,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,week
0,count,253,253,253,253,253,253
1,mean,2.0181078111027668E9,None,None,None,None,9.122529644268775
2,stddev,11449.014221731875,None,None,None,None,4.979801097667659
3,min,2018090600,09/06/2018,09:30:00,ARI,ARI,1
4,25%,2018100702,None,None,None,None,5
5,50%,2018110407,None,None,None,None,9
6,75%,2018120600,None,None,None,None,14
7,max,2018123015,12/30/2018,22:20:00,WAS,WAS,17


In [68]:
games_df.limit(10).toPandas()

,gameId,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,week
0,2018090600,09/06/2018,20:20:00,PHI,ATL,1
1,2018090901,09/09/2018,13:00:00,CLE,PIT,1
2,2018090902,09/09/2018,13:00:00,IND,CIN,1
3,2018090903,09/09/2018,13:00:00,MIA,TEN,1
4,2018090900,09/09/2018,13:00:00,BAL,BUF,1
5,2018090905,09/09/2018,13:00:00,NE,HOU,1
6,2018090907,09/09/2018,13:00:00,NYG,JAX,1
7,2018090906,09/09/2018,13:00:00,NO,TB,1
8,2018090909,09/09/2018,16:25:00,ARI,WAS,1
9,2018090910,09/09/2018,16:25:00,CAR,DAL,1


In [70]:
# read in players
players_df = sc.read.csv("../data/players.csv", inferSchema=True, header=True)
players_df.printSchema()

root
 |-- nflId: integer (nullable = true)
 |-- height: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- birthDate: string (nullable = true)
 |-- collegeName: string (nullable = true)
 |-- position: string (nullable = true)
 |-- displayName: string (nullable = true)



In [72]:
missing_plyrs = get_missing_vals_cnt(players_df)
distinct_plyrs = get_distinct_vals(players_df)
missing_plyrs.merge(distinct_plyrs, on='column', how='inner')

,column,n_missing,n_distinct
0,nflId,0,1303
1,height,0,29
2,weight,0,139
3,birthDate,0,1150
4,collegeName,0,251
5,position,0,21
6,displayName,0,1298


In [73]:
players_df.limit(10).toPandas()

,nflId,height,weight,birthDate,collegeName,position,displayName
0,2539334,72,190,1990-09-10,Washington,CB,Desmond Trufant
1,2539653,70,186,1988-11-01,Southeastern Louisiana,CB,Robert Alford
2,2543850,69,186,1991-12-18,Purdue,SS,Ricardo Allen
3,2555162,73,227,1994-11-04,Louisiana State,MLB,Deion Jones
4,2555255,75,232,1993-07-01,Minnesota,OLB,De'Vondre Campbell
5,2555543,73,216,1995-07-26,Florida,FS,Keanu Neal
6,2556445,70,211,1992-10-20,Florida,CB,Brian Poole
7,2507763,6-0,200,08/01/1986,Mississippi,WR,Mike Wallace
8,2532842,78,243,1989-01-20,Arizona,QB,Nick Foles
9,2540158,77,250,1990-11-10,Stanford,TE,Zach Ertz


In [87]:
players_df.filter(col("height").contains("-")).select("height").distinct().sort("height").show()

+------+
|height|
+------+
|  5-10|
|  5-11|
|   5-6|
|   5-7|
|   5-8|
|   5-9|
|   6-0|
|   6-1|
|   6-2|
|   6-3|
|   6-4|
|   6-5|
|   6-6|
|   6-7|
+------+



In [88]:
players_df = (players_df.withColumn("height", when(col("height") == '5-10', "70")
                                .when(col("height") == '6-0', '72')
                                .when(col("height") == '5-9', '69')
                                .when(col("height") == '6-2', '74')
                                .when(col("height") == '6-1', '73')
                                .when(col("height") == '6-6', '78')
                                .when(col("height") == '5-7', "67")
                                .when(col("height") == '6-3', '75')
                                .when(col("height") == '5-8', "68")
                                .when(col("height") == '5-6', "66")
                                .when(col("height") == '6-5', "77")
                                .when(col("height") == '6-4', "76")
                                .when(col("height") == '5-11', "71")
                                .when(col("height") == '6-7', "79")
                                .otherwise(col("height"))))
players_df = players_df.withColumn("height", col("height").cast(IntegerType()))

In [89]:
def toDate(col, formats=("yyyy-MM-dd", "MM/dd/yyyy")):
    """
    Converts multiple string dates into one format. 
    The function was adopted from https://stackoverflow.com/questions/46594750/cast-column-containing-multiple-string-date-formats-to-datetime-in-spark

    Args:
        col: Name of column to convert
        formats: List of date formats in the column

    Returns:
        Reformats dates so that they are a single format and casts to a DateType()
    """
    return coalesce(*[to_date(col, f) for f in formats])

In [90]:
players_df = players_df.withColumn("birthDate", toDate("birthDate"))

In [92]:
players_df.limit(10).toPandas()

,nflId,height,weight,birthDate,collegeName,position,displayName
0,2539334,72,190,1990-09-10,Washington,CB,Desmond Trufant
1,2539653,70,186,1988-11-01,Southeastern Louisiana,CB,Robert Alford
2,2543850,69,186,1991-12-18,Purdue,SS,Ricardo Allen
3,2555162,73,227,1994-11-04,Louisiana State,MLB,Deion Jones
4,2555255,75,232,1993-07-01,Minnesota,OLB,De'Vondre Campbell
5,2555543,73,216,1995-07-26,Florida,FS,Keanu Neal
6,2556445,70,211,1992-10-20,Florida,CB,Brian Poole
7,2507763,72,200,1986-08-01,Mississippi,WR,Mike Wallace
8,2532842,78,243,1989-01-20,Arizona,QB,Nick Foles
9,2540158,77,250,1990-11-10,Stanford,TE,Zach Ertz


In [163]:
num_college_players = players_df.groupBy("collegeName").agg(count("displayName").alias("numPlayers")).sort("numPlayers", ascending=False).toPandas()

In [164]:
bar = alt.Chart(num_college_players, title="Number of College Players in The NFL").mark_bar(color="steelblue").encode(alt.X("collegeName", title=""), alt.Y("numPlayers", title="Count"))
mean = alt.Chart(num_college_players).mark_rule(color='red').encode(y="mean(numPlayers)")
(bar + mean)

alt.LayerChart(...)

In [182]:
num_college_pos = players_df.groupBy("collegeName", "position").agg(count("displayName").alias("numPlayers")).na.fill(0).toPandas()

In [183]:
num_college_pos

,collegeName,position,numPlayers
0,Western Michigan,WR,1
1,Miami,DE,2
2,Rutgers,RB,1
3,"California, Pa.",CB,1
4,USC,CB,2
...,...,...,...
894,Vanderbilt,LS,1
895,Michigan,WR,2
896,UCLA,MLB,1
897,California-Davis,P,2


In [191]:
bar = alt.Chart(num_college_pos, title="Number of Players in NFL by College & Position").mark_bar().encode(alt.X("collegeName", title=""), alt.Y("numPlayers", title="Count"), color="position")

alt.Chart(...)

In [19]:
# read in plays
plays_df = sc.read.csv("../data/plays.csv", header=True, inferSchema=True)
plays_df.printSchema()

root
 |-- gameId: integer (nullable = true)
 |-- playId: integer (nullable = true)
 |-- playDescription: string (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- down: integer (nullable = true)
 |-- yardsToGo: integer (nullable = true)
 |-- possessionTeam: string (nullable = true)
 |-- playType: string (nullable = true)
 |-- yardlineSide: string (nullable = true)
 |-- yardlineNumber: integer (nullable = true)
 |-- offenseFormation: string (nullable = true)
 |-- personnelO: string (nullable = true)
 |-- defendersInTheBox: integer (nullable = true)
 |-- numberOfPassRushers: integer (nullable = true)
 |-- personnelD: string (nullable = true)
 |-- typeDropback: string (nullable = true)
 |-- preSnapVisitorScore: integer (nullable = true)
 |-- preSnapHomeScore: integer (nullable = true)
 |-- gameClock: string (nullable = true)
 |-- absoluteYardlineNumber: integer (nullable = true)
 |-- penaltyCodes: string (nullable = true)
 |-- penaltyJerseyNumbers: string (nullable = true)
 |-

19239